In [2]:
devtools::install_github("IHIW/pould/pould", build_vignettes = TRUE)
library(pould)
phased_haps <- read.delim("unrelated_haps.txt", sep="", header=TRUE)
#phased_haps <- read.delim("phased_haps.txt", sep="", header=TRUE)
nrow(phased_haps)
head(phased_haps)
phased_haps <- phased_haps[, c(1,2,3,6,7,4,5,14,15,12,13,10,11,8,9)]
head(phased_haps)
df <- phased_haps
 rows_with_zero <- df[apply(df, 1, function(row) any(row == 0)), ]

columns <- c("A_phased_allele_", "C_phased_allele_", "B_phased_allele_", "DRB1_phased_allele_", 
             "DQA1_phased_allele_", "DQB1_phased_allele_", "DPB1_phased_allele_")

nrow(phased_haps)
head(phased_haps)
# Define the columns and initialize the two lists
columns <- c("A_phased_allele_", "C_phased_allele_", "B_phased_allele_", "DRB1_phased_allele_", 
             "DQA1_phased_allele_", "DQB1_phased_allele_", "DPB1_phased_allele_")
names_upper_right <- list()
names_lower_left <- list()

# Loop through each pair of column names to create data frames
for (i in 1:(length(columns) - 1)) {
  for (j in (i + 1):length(columns)) {
    # Access the column names based on i and j
    col_name_i <- columns[i]
    col_name_j <- columns[j]
    
    # Find matching columns in phased_haps for each pair
    matching_cols_1 <- grep(paste0("^", col_name_i), colnames(phased_haps), value = TRUE)
    matching_cols_2 <- grep(paste0("^", col_name_j), colnames(phased_haps), value = TRUE)
    selected_cols <- c(matching_cols_1, matching_cols_2)
    
    # Get short names for columns
    name1 <- strsplit(col_name_i, split="_")[[1]][1]
    name2 <- strsplit(col_name_j, split="_")[[1]][1]
    
    # Extract the restricted column data frame
    df <- phased_haps[, selected_cols]
    colnames(df) <- c(name1, name1, name2, name2)  # Rename columns for simplicity
    
    # Assign the data frame to a unique name and store it in the appropriate list
    pair_name <- paste0(col_name_i, col_name_j)
    
    # For the upper right matrix portion
    assign( pair_name, df)
    names_upper_right <- c(names_upper_right,pair_name)
    
    # For the lower left matrix portion
    pair_name_reversed <- paste0(col_name_j, col_name_i)
    assign(pair_name_reversed, df)
    names_lower_left <- c(names_lower_left, pair_name_reversed)
  }
}

LD_matrix <- matrix(NA, nrow = 7, ncol = 7)
colnames(LD_matrix) <- c("A","C","B","DRB1","DQA1","DQB1","DPB1")
rownames(LD_matrix) <- c("DPB1","DQB1","DQA1","DRB1","B","C","A")

##upper right
for(i in names_upper_right){
  print(i)
  df <- get(i)
  df <- df[apply(df, 1, function(row) !any(row == 0)), ]
  result <- pould::cALD(df, inPhase=TRUE, verbose=FALSE)
  print(result)
  row <- strsplit(i, split="_")[[1]][1]
  print(row)
  col <- strsplit(i, split="_")[[1]][4]
  print(col)
  #LD_matrix[row, col] <-as.numeric(result[4])
  LD_matrix[col, row]  <- as.numeric(result[3])
  }
###lower right
for(i in names_lower_left){
  print(i)
  df <- get(i)
  df <- df[apply(df, 1, function(row) !any(row == 0)), ]
  result <- pould::cALD(df, inPhase=TRUE, verbose=FALSE)
  print(result)
  row <- strsplit(i, split="_")[[1]][1]
  print(row)
  col <- strsplit(i, split="_")[[1]][4]
  print(col)
  #LD_matrix[row, col] <-as.numeric(result[4])
  LD_matrix[col, row]  <- as.numeric(result[4])
}


library(ggplot2)
library(tidyr)  # For melting the matrix


LD_matrix_df <- as.data.frame(LD_matrix)
LD_matrix_df$Row <- rownames(LD_matrix)
LD_matrix_df <- tidyr::pivot_longer(LD_matrix_df, -Row, names_to = "Column", values_to = "value")

# Factor levels to ensure order aligns with matrix structure
LD_matrix_df$Row <- factor(LD_matrix_df$Row, levels = rownames(LD_matrix))
LD_matrix_df$Column <- factor(LD_matrix_df$Column, levels = colnames(LD_matrix))

# Plot directly from the matrix values
ggplot(LD_matrix_df, aes(x = Column, y = Row)) +
  geom_tile(aes(fill = value), color = "white") +  # Tile for each matrix cell
  geom_text(aes(label = round(value, 2)), color = "black") +  # Display values directly
  scale_fill_gradient(low = "white", high = "red") +  # Color gradient for LD values
  theme_minimal() +
  theme(
    axis.title.x = element_blank(),
    axis.title.y = element_blank(),
    axis.text.x = element_text(hjust = 1, size = 12),  # Adjust x-axis text
    axis.text.y = element_text(size = 12),  # Set y-axis text size
    panel.grid = element_blank(),  # Remove grid lines
    axis.line.x = element_blank()  # Remove x-axis line for a cleaner look
  ) +
  labs(fill = "LD Value") +
  scale_y_discrete(position = "left") +  # Y-axis on the right side
  scale_x_discrete(position = "top")  +
 ggtitle("Asymmetric Linkage Disequilibrium: row gene conditional on column gene")



[1] 102

,sample.id,A_phased_allele_1,A_phased_allele_2,B_phased_allele_1,B_phased_allele_2,C_phased_allele_1,C_phased_allele_2,DPB1_phased_allele_1,DPB1_phased_allele_2,DQB1_phased_allele_1,DQB1_phased_allele_2,DQA1_phased_allele_1,DQA1_phased_allele_2,DRB1_phased_allele_1,DRB1_phased_allele_2,full_hap,class1_hap,class2_hap,hap1,hap2
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<chr>,<chr>
1,3,30:02,33:03,15:10,81:01,03:04,08:04,01:01,01:01,02:01,06:02,05:01,01:02,03:01,12:02,1,1,1,A30:02-C03:04-B15:10-DRB103:01-DQA105:01-DQB102:01-DPB101:01,A33:03-C08:04-B81:01-DRB112:02-DQA101:02-DQB106:02-DPB101:01
2,15,30:01,30:02,42:01,45:01,17:01,16:01,01:01,04:02,02:01,02:01,03:01,02:01,09:01,07:01,1,1,1,A30:01-C17:01-B42:01-DRB109:01-DQA103:01-DQB102:01-DPB101:01,A30:02-C16:01-B45:01-DRB107:01-DQA102:01-DQB102:01-DPB104:02
3,16,68:02,23:01,41:01,41:01,08:02,08:02,04:02,02:01,06:02,06:02,01:02,01:02,08:04,11:01,1,1,1,A68:02-C08:02-B41:01-DRB108:04-DQA101:02-DQB106:02-DPB104:02,A23:01-C08:02-B41:01-DRB111:01-DQA101:02-DQB106:02-DPB102:01
4,26,30:01,23:01,53:01,41:01,04:01,08:02,04:02,02:01,02:01,06:02,05:01,01:02,03:01,11:01,1,1,1,A30:01-C04:01-B53:01-DRB103:01-DQA105:01-DQB102:01-DPB104:02,A23:01-C08:02-B41:01-DRB111:01-DQA101:02-DQB106:02-DPB102:01
5,30,01:01,68:02,81:01,53:01,18:01,04:01,01:01,01:01,03:01,06:04,05:01,01:02,11:01,13:02,1,1,1,A01:01-C18:01-B81:01-DRB111:01-DQA105:01-DQB103:01-DPB101:01,A68:02-C04:01-B53:01-DRB113:02-DQA101:02-DQB106:04-DPB101:01
6,32,30:01,80:01,42:01,18:01,17:01,02:02,18:01,04:02,03:01,02:01,05:01,02:01,11:02,07:01,1,1,1,A30:01-C17:01-B42:01-DRB111:02-DQA105:01-DQB103:01-DPB118:01,A80:01-C02:02-B18:01-DRB107:01-DQA102:01-DQB102:01-DPB104:02


,sample.id,A_phased_allele_1,A_phased_allele_2,C_phased_allele_1,C_phased_allele_2,B_phased_allele_1,B_phased_allele_2,DRB1_phased_allele_1,DRB1_phased_allele_2,DQA1_phased_allele_1,DQA1_phased_allele_2,DQB1_phased_allele_1,DQB1_phased_allele_2,DPB1_phased_allele_1,DPB1_phased_allele_2
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,3,30:02,33:03,03:04,08:04,15:10,81:01,03:01,12:02,05:01,01:02,02:01,06:02,01:01,01:01
2,15,30:01,30:02,17:01,16:01,42:01,45:01,09:01,07:01,03:01,02:01,02:01,02:01,01:01,04:02
3,16,68:02,23:01,08:02,08:02,41:01,41:01,08:04,11:01,01:02,01:02,06:02,06:02,04:02,02:01
4,26,30:01,23:01,04:01,08:02,53:01,41:01,03:01,11:01,05:01,01:02,02:01,06:02,04:02,02:01
5,30,01:01,68:02,18:01,04:01,81:01,53:01,11:01,13:02,05:01,01:02,03:01,06:04,01:01,01:01
6,32,30:01,80:01,17:01,02:02,42:01,18:01,11:02,07:01,05:01,02:01,03:01,02:01,18:01,04:02


[1] 102

,sample.id,A_phased_allele_1,A_phased_allele_2,C_phased_allele_1,C_phased_allele_2,B_phased_allele_1,B_phased_allele_2,DRB1_phased_allele_1,DRB1_phased_allele_2,DQA1_phased_allele_1,DQA1_phased_allele_2,DQB1_phased_allele_1,DQB1_phased_allele_2,DPB1_phased_allele_1,DPB1_phased_allele_2
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,3,30:02,33:03,03:04,08:04,15:10,81:01,03:01,12:02,05:01,01:02,02:01,06:02,01:01,01:01
2,15,30:01,30:02,17:01,16:01,42:01,45:01,09:01,07:01,03:01,02:01,02:01,02:01,01:01,04:02
3,16,68:02,23:01,08:02,08:02,41:01,41:01,08:04,11:01,01:02,01:02,06:02,06:02,04:02,02:01
4,26,30:01,23:01,04:01,08:02,53:01,41:01,03:01,11:01,05:01,01:02,02:01,06:02,04:02,02:01
5,30,01:01,68:02,18:01,04:01,81:01,53:01,11:01,13:02,05:01,01:02,03:01,06:04,01:01,01:01
6,32,30:01,80:01,17:01,02:02,42:01,18:01,11:02,07:01,05:01,02:01,03:01,02:01,18:01,04:02


[1] "A_phased_allele_C_phased_allele_"


ERROR: Error in loadNamespace(x): there is no package called ‘pould’
